## **概括**

目前还有宝箱与Buff的奖励仍未加入，为了使agent的得分尽可能的高，需要在到达终点的前提下，尽量收集完所有宝箱，同时为了使模型有一个比较好的能力，采用分阶段训练，第一阶段先训练模型到达终点，随后进行融合，在到达终点的同时考虑宝箱的收集，最后阶段完全考虑宝箱的收集。

特征计算方面，引入了最近宝箱和buff的特征14维(格式同终点)，4维的额外信息(步数比例，宝箱收集比例，每步得分，buff收集，7*7障碍密度)，以及多宝箱--终点联合特征(最近的3个宝箱到终点的距离以及角度)。同时为了使模型更好的找到宝箱，加快收敛速度，引入了潜势函数。

### **特征维度**

1. 新引入的14维宝箱与buff特征，均调用`_get_pos_feature`函数，内部具体特征同之前

```python
        # 奖励检测
        cur_treasure_count = obs["score_info"]["treasure_collected_count"]
        cur_buff_count = obs["score_info"]["buff_count"]
        self.treasure_gain = cur_treasure_count - self.prev_treasure_count
        self.buff_gain = cur_buff_count - self.prev_buff_count
        self.prev_treasure_count = cur_treasure_count
        self.prev_buff_count = cur_buff_count

        # 寻找宝箱
        visible_treasures = [org for org in obs["frame_state"]["organs"] 
                              if org["sub_type"] == 1 and org["status"] == 1]
        self.visible_treasures = [
            (org["pos"]["x"], org["pos"]["z"]) for org in visible_treasures
        ]
        if visible_treasures:
            # 最近宝箱
            nearest = min(visible_treasures, key=lambda o: 
                          (o["pos"]["x"]-self.cur_pos[0])**2 + (o["pos"]["z"]-self.cur_pos[1])**2)
            target = (nearest["pos"]["x"], nearest["pos"]["z"])
            self.feature_treasure = self._get_pos_feature(1, self.cur_pos, target)
            self.cur_treasure_dist = np.linalg.norm(np.array(self.cur_pos) - np.array(target))
        else:
            # 未发现宝箱
            self.feature_treasure = np.concatenate([
                [0.0], np.zeros(8), np.zeros(2), np.zeros(2), [1.0]
            ], dtype=np.float32)
            self.cur_treasure_dist = None
        # 寻找Buff
        buff_obj = next((org for org in obs["frame_state"]["organs"] 
                         if org["sub_type"] == 2 and org["status"] == 1), None)
        if buff_obj:
            bpos = (buff_obj["pos"]["x"], buff_obj["pos"]["z"])
            self.feature_buff = self._get_pos_feature(1, self.cur_pos, bpos)
        else:
            self.feature_buff = np.concatenate([
                [0.0], np.zeros(8), np.zeros(2), np.zeros(2), [1.0]
            ], dtype=np.float32)
```


2. 额外信息

- 分数信息，`self.score_delta`用于表征当前步数的得分变化
```python
cur_score = obs["score_info"]["score"]
self.score_delta = self.prev_score - cur_score
self.prev_score = cur_score
```

- 步数，宝箱，buff全局信息，注意使用归一化，传入`extra_feats`中
```python
step_ratio = obs["score_info"]["step_no"] /  Config.MAX_STEP
treasure_ratio = obs["score_info"]["treasure_collected_count"] / Config.TOTAL_TREASURES
score_delta = self.score_delta
talent_count = obs["score_info"]["talent_count"] / 100
```

- 7*7障碍密度信息
```python
h, w = self.local_map.shape
center_y, center_x = h // 2, w // 2
r = 3
y0 = max(0, center_y - r)
y1 = min(h, center_y + r + 1)
x0 = max(0, center_x - r)
x1 = min(w, center_x + r + 1)
submap = self.local_map[y0:y1, x0:x1]
if submap.size > 0:
    density_7x7 = float(np.mean(submap != 0))
else:
    density_7x7 = 0.0
    extra_feats.append(density_7x7)
```


3. 宝箱终点联合信息，返回归一化的两个距离以及角度
```python
cx, cz = self.cur_pos
t2e_feats = []
# 距离排序
vt = sorted(self.visible_treasures,
            key=lambda p: (p[0]-cx)**2+(p[1]-cz)**2)[:3]
for bx, bz in vt:
    dx1, dz1 = bx-cx, bz-cz
    dx2, dz2 = self.end_pos[0]-bx, self.end_pos[1]-bz if self.end_pos else (0,0)
    d1_norm = math.hypot(dx1, dz1)/self.max_map_dist
    d2_norm = math.hypot(dx2, dz2)/self.max_map_dist
    cos_ang = (dx1*dx2 + dz1*dz2) / (math.hypot(dx1,dz1)*math.hypot(dx2,dz2)+1e-6)
    t2e_feats += [d1_norm, d2_norm, cos_ang]
while len(t2e_feats)<9: t2e_feats.append(0.0)
```



### **潜势函数**

设计势函数$\varPhi \left( s \right) =-d$，得到
$$shape\ =\ \gamma \varPhi \left( s \right) -\varPhi \left( s^{'} \right) $$
其中$\varPhi \left( s \right)$为当前状态的势能，$\varPhi \left( s^{'} \right)$为前一状态的势能，给宝箱和终点均设计潜势函数，可以引导agent向宝箱/终点靠近。(注意归一化)

同时为了能控制分阶段训练，设计$t,e$权重值加权，最终势能公式
$$shape = t \times shape_t + e \times shape_e$$

具体代码如下：
```python
        shape_T = 0.0
        if self.cur_treasure_dist is not None:
            cur_td_norm = self.cur_treasure_dist / self.max_map_dist
            cur_phi_t = -cur_td_norm
            prev_phi_t = getattr(self, "prev_treasure_phi", 0.0)
            shape_T     = Config.TREASURE_REWARD * (gamma_0 * cur_phi_t - prev_phi_t)
            self.prev_treasure_phi = cur_phi_t
        else:
            self.prev_treasure_phi = 0.0
        
        shape_E = 0.0
        if self.end_pos is not None:
            raw_ed      = np.linalg.norm(np.array(self.cur_pos) - np.array(self.end_pos))
            cur_ed_norm = raw_ed / self.max_map_dist
            cur_phi_e   = -cur_ed_norm
            prev_phi_e  = getattr(self, "prev_end_phi", 0.0)
            shape_E     = Config.GOAL_REWARD * (gamma_0 * cur_phi_e - prev_phi_e)
            self.prev_end_phi = cur_phi_e
        else:
            self.prev_end_phi = 0.0

        shape = t * shape_T + e * shape_E
```

分别针对宝箱和终点设计一次性收集奖励和到达奖惩(t,e)均为上述的权重。此处为了使agent能尽可能手机玩宝箱，给出一个到达终点但是未收集完宝箱的惩罚 
```python
        one_time = Config.TREASURE_IMMEDIATE_REWARD * max(0, self.treasure_gain) * t
        if done:
            # 宝箱训练中，未收集完宝箱到终点
            if t > 0.0 and obs["score_info"]["treasure_collected_count"] < Config.TOTAL_TREASURES:
                end_pen = Config.INCOMPLETE_END_PENALTY * t
            # 宝箱训练中，收集完宝箱到终点
            elif t > 0.0 and obs["score_info"]["treasure_collected_count"] == Config.TOTAL_TREASURES:
                end_pen = Config.GOAL_REWARD * e + Config.PERFECT_REWARD * e
            # 终点训练奖励
            else:
                end_pen = Config.GOAL_REWARD * e
        else:
            end_pen = 0.0
```

同样的buff奖励也需要考虑，收集`buff_gain * 0.2`，可以调节0.2参数进行设置。




### **分阶段训练**

为了使agent能更好的学会即收集宝箱又到达终点，采用分阶段训练策略，如概述所说，一阶段先训练模型到达终点，随后进行融合，在到达终点的同时考虑宝箱的收集，最后阶段完全考虑宝箱的收集。故需要对终点和宝箱的奖励进行加权，使用上述的$t, e$参数，分别代表宝箱权重和终点权重。

在第一阶段，追逐终点$t=0.0, e=1.0$，第二阶段，开始动态融合，第三阶段$t=1.0, e=0.0$，同时为了防止在最后阶段agent完全忽视终点的问题，对参数$e$进行兜底，即$e=max(e, E_MIN)$，同时需要保证$t+e=1$具体代码如下：其中`Config.S1_STEPS`为阶段1的学习次数，`Config.S2_STEPS`为阶段2的学习次数(学习次数指的是agent进行learn的次数，具体在`train_workflow`中详细使用)
```python
if global_step < Config.S1_STEPS:
    e, t = 1.0, 0.0
elif global_step < Config.S2_STEPS:
    e = 1 - (global_step - Config.S1_STEPS) / (Config.S2_STEPS - Config.S1_STEPS)
else:
    e, t= 0.0, 1.0
gamma_0 = 0.99
e = max(e, Config.E_MIN)
treasure_left = Config.TOTAL_TREASURES - obs["score_info"]["treasure_collected_count"]
if treasure_left == 0:
     e = 1.0
elif treasure_left <= 2:
    e = max(e, 0.6)
t = 1 - e
shape = t * shape_T + e * shape_E  
```

同时注意需要修改前面设计的终点相关奖励，传入一个$e$进行加权，防止影响
```python
near_goal_penalty = near_goal_penalty * e
cone_reward = 0.3 * (0.3 - end_dist) if end_dist < 0.3 else 0.0
cone_reward = cone_reward * e
```

分阶段训练使用的轮数放在了`train_workflow.py`文件中，新增`agent.global_step=0`计数，没学习一次则+1，如下(我设置的`episode_num_every_epoch`==1)，即每一局学习一次并且次数+1，在`obseration_process`中增加传入参数`agent.global_step`和`done`，进入传入`process()`中进行计算与分析。具体见`preprocessor.py`文件中的`process`函数。需要注意的是，在`exploit`中也使用了`observation_process`函数，但是由于评估模式下，不需要使用奖励设计，所以可以随便传入一个`agent.global_step`和`done`即可，否则会出现测试异常。
```python
while True:
    for g_data, monitor_data in run_episodes(episode_num_every_epoch, env, agent,     usr_conf, logger, monitor, epoch):
        agent.learn(g_data)
        agent.global_step += 1
        g_data.clear()
```

最后本版本的代码在监控数据中增加了权重$t$和$e$的具体数值，可以看到情况，以及`agent.global_step`的具体轮数，方便查看。

### **配置文件**

为了统一参数的使用，部分需要更改调整的参数放在了`conf/conf.py`文件下
```python
# 训练调度
S1_STEPS = 2500
S2_STEPS = 8000
MAX_STEP = 2000

# 宝箱
TOTAL_TREASURES = 8

# 潜势函数
TREASURE_REWARD = 2.5  # 潜势函数使用
TREASURE_IMMEDIATE_REWARD = 2.5 # 收集宝箱立刻给

GOAL_REWARD = 5.0
INCOMPLETE_END_PENALTY = -3.5
PERFECT_REWARD = 2.5

REWARD_CLIP = 5.0

E_MIN = 0.25
```



### **问题**

现在的核心问题还是在：agent要么能走到终点收集2-3个少量的宝箱，要么收集大量的6-8个宝箱，但是无法走到终点，两者很难做到平衡

优化思路：
1. 可以考虑做更多的特征维度，宝箱收集信息等，局部视野情况，可以参考社区的文章
2. 继续调整参数，平衡宝箱和终点之间的收益，
3. 模型可以继续优化修改(当前的reward和q_vlaues)仍然存在震荡，多少局学习也可以进行修改